In [1]:
import json
import networkx as nx
import random
import matplotlib.pyplot as plt
import math
log = math.log2

In [124]:
def import_channel_graph():
    # retrieve this by: lightning-cli listchannels > listchannels.json
    f = open("listchannels.json")
    jsn = json.load(f)
    G = nx.Graph()
    for channel in jsn["channels"]:
        src = channel["source"]
        dest = channel["destination"]
        cap = int(int(channel["satoshis"])/1000)
        sid = channel["short_channel_id"]
        G.add_edge(src,dest,weight=cap,sid=sid)
    return G

channel_graph = import_channel_graph()

In [125]:
channel_graph = nx.Graph()
channel_graph.add_edge("S","A",capacity=2)
#channel_graph.add_edge("A0","A",capacity=2)
channel_graph.add_edge("S","X",capacity=1)
channel_graph.add_edge("A","B",capacity=2)
channel_graph.add_edge("X","B",capacity=9)
channel_graph.add_edge("X","Y",capacity=7)
channel_graph.add_edge("Y","D",capacity=4)
channel_graph.add_edge("B","D",capacity=4)


In [126]:
nx.minimum_cut(channel_graph, "B", "Y")

(11, ({'A', 'B', 'D', 'S', 'X'}, {'Y'}))

In [149]:
def generate_null_flow(channel_graph):
    flow = {n:{} for n in channel_graph.nodes()}
    for u,v in channel_graph.edges():
        flow[u][v]=0
        flow[v][u]=0
    return flow

def compute_flow_amount(flow,src):
    return sum(v for k,v in flow[src].items())

def compute_probability_of_flow(flow, channel_graph):
    res = 1
    for u,v in channel_graph.edges():
        capacity = channel_graph[u][v]["capacity"]
        forward = flow[u][v]
        backward = flow[v][u]
        prob = (capacity+1-abs(forward-backward))/(capacity+1)
        res*=prob
    return res
            
#tricky because of directions but it shouldn't be too hard
def compute_residual_graph(channel_graph,flow,amt):
    residual = nx.DiGraph()
    for u,v in channel_graph.edges():
        forward_flow = flow[u][v]
        backward_flow = flow[v][u]
        channel_capacity = channel_graph[u][v]["capacity"]
        
        if forward_flow > 0 and backward_flow > 0:
            settle = min(forward_flow, backward_flow)
            flow[u][v]-=settle
            forward_flow -= settle
            flow[v][u]-=settle
            backward_flow -= settle
        
        
        if forward_flow == 0 and backward_flow == 0 and amt <= channel_capacity:
            prob = (channel_capacity + 1 - amt)/(channel_capacity + 1)
            residual.add_edge(u,v,weight=-1*log(prob))
            residual.add_edge(v,u,weight=-1*log(prob))
        else:
            if forward_flow > 0:
                #P(X >= f + amt | X >= f) = (c+1-f-a)/(c+1-f)
                if forward_flow+amt <= channel_capacity:
                    prob = (channel_capacity + 1 - forward_flow - amt) / (channel_capacity - forward_flow + 1)
                    residual.add_edge(u,v,weight=-1*log(prob))
                if amt <= forward_flow:
                    # 1/ P(X>=f | X >= f - amt) = (c+1-f)/(c+1-f+a)
                    inverse_prob = (channel_capacity+1-forward_flow + amt)/(channel_capacity+1-forward_flow)
                    residual.add_edge(v,u,weight=-log(inverse_prob))
            elif backward_flow > 0:
                if backward_flow + amt <= channel_capacity:
                    prob = (channel_capacity - backward_flow + 1 - amt)/(channel_capacity + 1 - backward_flow)
                    residual.add_edge(v,u,weight=-log(prob))
                if amt <= backward_flow:
                    # 1/ P(X>=f | X >= f - amt) = (c+1-f)/(c+1-f+a)
                    inverse_prob = (channel_capacity+1-backward_flow+amt)/(channel_capacity+1-backward_flow)
                    residual.add_edge(u,v,weight=-log(inverse_prob))
    return residual
                

In [150]:
def next_hop(path):
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)
        
def update_flow(flow,path,amt):
    for src,dest in next_hop(path):
        flow[src][dest]+=amt

def print_flow(flow):
    for src, value in flow.items():
        for dest, f in value.items():
            if f > 0:
                print(src,dest,f)
        
#update_flow(flow,path,1)

In [153]:
S = "S"
D = "D"

flow=generate_null_flow(channel_graph)
print(compute_probability_of_flow(flow,channel_graph))
    
for a in range(1,nx.minimum_cut(channel_graph, S, D)[0]+1):
    res = compute_residual_graph(channel_graph,flow,1)
    for u,v in res.edges():
        print(u,v,2**-res[u][v]["weight"])
    path = nx.bellman_ford_path(res,S,D,weight="weight")
    print(path)
    update_flow(flow,path,1)
    print_flow(flow)
    print(compute_probability_of_flow(flow,channel_graph))


1.0
S A 0.6666666666666666
S X 0.5
A S 0.6666666666666666
A B 0.6666666666666666
X S 0.5
X B 0.9
X Y 0.875
B A 0.6666666666666666
B X 0.9
B D 0.8
Y X 0.875
Y D 0.8
D B 0.8
D Y 0.8
['S', 'X', 'B', 'D']
S X 1
X B 1
B D 1
0.36000000000000004
S A 0.6666666666666666
A S 0.6666666666666666
A B 0.6666666666666666
X S 2.0
X B 0.8888888888888888
X Y 0.875
B A 0.6666666666666666
B X 1.1111111111111112
B D 0.75
Y X 0.875
Y D 0.8
D B 1.25
D Y 0.8
['S', 'A', 'B', 'X', 'Y', 'D']
S A 1
S X 1
A B 1
X B 1
X Y 1
B X 1
B D 1
Y D 1
0.12444444444444445
S A 0.5
A S 1.5
A B 0.5
X S 2.0
X B 0.9
X Y 0.8571428571428571
B A 1.5
B X 0.9
B D 0.75
Y X 1.1428571428571428
Y D 0.75
D B 1.25
D Y 1.25
['S', 'A', 'B', 'D']
S A 2
S X 1
A B 2
X Y 1
B D 2
Y D 1
0.02333333333333333


In [46]:
for u,v in res.edges():
    print(u,v,2**-res[u][v]["weight"])

S A 0.6666666666666666
S X 0.5
A S 0.6666666666666666
A B 0.6666666666666666
X S 0.5
X B 0.99
X Y 0.875
B A 0.6666666666666666
B X 0.99
B D 0.8
Y X 0.875
Y D 0.8
D B 0.8
D Y 0.8


In [ ]:
res = 1
for s,d in path:
    res*=